# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/milestone_202110'))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
# DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
VISUALIZE = False

# Tool dimensions
TOOL_DIM = [0.32, 0.08]
TOOL_OFFSET = 0.0
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

ip_cur = "192.168.0.7"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

from utils.streaming import *
from utils.detection_util import *

if CONNECT_MOBILE:
    sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)
    time.sleep(1)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,0)),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.set_workspace_boundary(-3, 7, -5, 5, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
# add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

add_kiro_indytool_down(gscene, zoff=TOOL_OFFSET, tool_link=TIP_LINK, face_name=TOOL_NAME)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2-CLEARANCE), rpy=(0,0,0))

indy = crob.robot_dict["indy1"]

Current PC IP: 192.168.0.7
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10
connection command:
kmb0: False
indy1: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


### 0.2 Wait task start queue

## 1. Bed cleaning

### 1.1 Detect bed

#### 1.1.1 Move to bed-seek pose 

In [2]:
VIEW_POSE = np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
    
if VISUALIZE:
    gscene.show_pose(VIEW_POSE_EXT)

 * Environment: production
   Use a production WSGI server instead.


#### 1.1.2  detect bed and add to the scene

In [3]:
## add default bed and closet
bed_center = (2,0,0)
bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)
bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

closet_leftup, closet_rightup, closet_down = add_closet(
    gscene, closet_center=np.matmul(Rot_rpy(bed_rpy), (-0.75,-1.5,0))+bed_center, closet_rpy=bed_rpy, 
    COLOR_CLOSET_COL=(0,1,0,0.3))

 * Debug mode: off


In [4]:
CLOSET_LOCATION = "LEFT"

T_bm_from = wayframer.get_tf_handle(list2dict(VIEW_POSE_EXT, gscene.joint_names))
T_bs = bed_mat.get_tf(VIEW_POSE_EXT)

if CLOSET_LOCATION == "LEFT":
    T_sm = SE3(Rot_axis(3, np.pi), [1.5, -1.33, 0])
elif CLOSET_LOCATION == "RIGHT":       
    T_sm = SE3(Rot_axis(3, np.pi), [1.5, 1.33, 0])
    
T_bm = np.matmul(T_bs, T_sm)

x,y = T_bm[:2,3]
theta = Rot2axis(T_bm[:3,:3], 3)
VIEW_MOVED_EXT = np.add(VIEW_POSE_EXT, [x,y,theta]+[0]*9) 

### 1.2 Make bed cleaning plan

#### 1.2.1 get division-base pose combination data

In [5]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [6]:
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
ccheck = CachedCollisionCheck(gcheck, wp_task, wp_hdl, wayframer)

BED_OFFSET = 0.15
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(0,0,-gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE-BED_OFFSET), 
                                  rpy=(0,0,0))

T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_HEIGHT = round(bed_mat.get_tf(HOME_DICT)[2,3] + bed_mat.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]

In [7]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(bed_mat, brush_face, robot_config, 
                                          plane_val=EE_HEIGHT, 
                                          tip_dir=SweepDirections.front.name, 
                                          TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02, 
                                          sweep_margin=0.0, xout_cut=True)

('Height Reference: ', 0.7020000109672546)


In [8]:
# import matplotlib.pyplot as plt
# XY = []
# UV = []
# for Tbm in Tbm_succ_all:
#     XY.append(Tbm[:2,3])
#     UV.append(np.matmul(Tbm[:2,:2], [0.1, 0]))
# XY = np.transpose(XY)
# UV = list(np.transpose(UV))
# plt.quiver(XY[0], XY[1], UV[0], UV[1])
# plt.quiver([Tbs[0,3]], [Tbs[1,3]], [Tbs[0,0]*0.1], [Tbs[0,1]*0.1])

#### 1.2.2 select base poses and generate motions

In [9]:
HOME_POSE_MOVE = VIEW_POSE
HOME_POSE_SWEEP = VIEW_MOVED_EXT
HOME_POSE_SWEEP[6:] = HOME_POSE_MOVE
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    
test_fun = TestBaseDivFunc(ppline, floor_ws, bed_mat, WP_DIMS, TOOL_DIM, crob.home_dict)

test_fun.clear()

idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step,
    test_fun=test_fun)

terminate_on_first is deprecated. Use max_solution_count=1
Use 20/20 agents
======================= terminated 16: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 18: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 2: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 7: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 4: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 1: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 6: max iteration time reached (1634826761/1634826758.74 s) ===============================
======================= terminated 9: ma

======================= terminated 7: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 19: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 6: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 15: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 10: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 17: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 18: max iteration time reached (1634826765/1634826762.38 s) ===============================
======================= terminated 14: max iteration time reached (1634826765/1634826762.38 s) ==================

======================= terminated 17: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 5: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 3: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 13: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 7: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 15: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 10: max iteration time reached (1634826769/1634826766.07 s) ===============================
======================= terminated 9: max iteration time reached (1634826769/1634826766.07 s) ====================

======================= terminated 1: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 17: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 19: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 8: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 12: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 11: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 7: max iteration time reached (1634826773/1634826769.77 s) ===============================
======================= terminated 14: max iteration time reached (1634826773/1634826769.77 s) ===================

======================= terminated 17: max iteration time reached (1634826776/1634826773.42 s) ===============================
======================= terminated 5: max iteration time reached (1634826776/1634826773.42 s) ===============================
======================= terminated 16: max iteration time reached (1634826776/1634826773.42 s) ===============================
======================= terminated 12: max iteration time reached (1634826776/1634826773.42 s) ===============================
======================= terminated 11: max iteration time reached (1634826776/1634826773.42 s) ===============================
======================= terminated 3: max iteration time reached (1634826776/1634826773.42 s) ===============================
terminate_on_first is deprecated. Use max_solution_count=1
Use 20/20 agents
======================= terminated 10: max iteration time reached (1634826779/1634826776.84 s) ===============================
======================= terminated 16

...

======================= terminated 5: required answers acquired ===============================
======================= terminated 8: required answers acquired from other agent ===============================
======================= terminated 12: required answers acquired from other agent ===============================
======================= terminated 2: required answers acquired from other agent ===============================
======================= terminated 4: required answers acquired from other agent ===============================
======================= terminated 3: required answers acquired from other agent ===============================
======================= terminated 6: required answers acquired from other agent ===============================
======================= terminated 19: required answers acquired from other agent ===============================
======================= terminated 13: required answers acquired from other agent =============================

======================= terminated 16: required answers acquired from other agent ===============================
Goal reached
======================= terminated 18: required answers acquired ===============================
======================= terminated 3: required answers acquired from other agent ===============================
Goal reached
======================= terminated 2: required answers acquired ===============================
Goal reached
======================= terminated 11: required answers acquired ===============================
Goal reached
======================= terminated 0: required answers acquired ===============================
Goal reached
======================= terminated 10: required answers acquired ===============================
terminate_on_first is deprecated. Use max_solution_count=1
Use 20/20 agents
Goal reached
Goal reached
Goal reached
======================= terminated 3: required answers acquired ===============================
Goal reached
=

Goal reached
======================= terminated 4: required answers acquired ===============================
======================= terminated 6: required answers acquired from other agent ===============================
terminate_on_first is deprecated. Use max_solution_count=1
Use 20/20 agents
======================= terminated 8: max iteration time reached (1634826794/1634826791.31 s) ===============================
======================= terminated 11: max iteration time reached (1634826794/1634826791.31 s) ===============================
======================= terminated 19: max iteration time reached (1634826794/1634826791.31 s) ===============================
======================= terminated 10: max iteration time reached (1634826794/1634826791.31 s) ===============================
======================= terminated 5: max iteration time reached (1634826794/1634826791.31 s) ===============================
======================= terminated 7: max iteration time reached (163

======================= terminated 5: required answers acquired from other agent ===============================
======================= terminated 4: required answers acquired from other agent ===============================
======================= terminated 9: required answers acquired from other agent ===============================
======================= terminated 12: required answers acquired from other agent ===============================
======================= terminated 16: required answers acquired from other agent ===============================
======================= terminated 8: required answers acquired from other agent ===============================
======================= terminated 13: required answers acquired from other agent ===============================
======================= terminated 11: required answers acquired from other agent ===============================
======================= terminated 15: required answers acquired from other agent ==========

Goal reached
Goal reached
======================= terminated 8: required answers acquired ===============================
======================= terminated 16: required answers acquired ===============================
======================= terminated 4: required answers acquired from other agent ===============================
Goal reached
======================= terminated 19: required answers acquired ===============================
Goal reached
======================= terminated 15: required answers acquired ===============================
Goal reached
======================= terminated 18: required answers acquired ===============================
======================= terminated 9: required answers acquired from other agent ===============================
======================= terminated 12: required answers acquired from other agent ===============================
Goal reached
======================= terminated 6: required answers acquired ===============================
Go

#### 1.2.3 refine motions

In [10]:
snode_schedule_list, scene_args_list, scene_kwargs_list = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, HOME_POSE_SWEEP, 
    floor_ws, wayframer, bed_mat, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached


### 1.3 Execute bed cleaning sequence

In [19]:
D_APPROACH = 0.4
UPDATE_MOTION = False
VEL_LEVEL = 1

with indy:
    vel_level_bak = indy.get_joint_vel_level()
    print("vel_level_bak: {}".format(vel_level_bak))
    
with indy:
    indy.set_joint_vel_level(VEL_LEVEL)

Connect: Server IP (192.168.0.3)
vel_level_bak: 5
Connect: Server IP (192.168.0.3)


In [20]:
swp_fin_list = []

for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    print("motions: {}".format(len(snode_schedule[:-1])-1))
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        snode_pre = snode_pre.copy(pscene)
        snode_pre.traj = None
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)
        print(subjects)

        if len(subjects) ==0 or subjects[0] == "sweep":
            to_state.Q[:6] = from_state.Q[:6]
            if CONNECT_INDY:
                if UPDATE_MOTION:
                    print("try update trajectory")
                    traj, state_next, error, succ = \
                            ppline.test_connection(from_state=snode_pre.state, 
                                                   to_state=snode_nxt.state)
                    if succ:
                        snode_nxt.traj = traj
                        snode_nxt.state = state_next
            #         else:
            #             raise(RuntimeError("Path update fail"))
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt])

        elif subjects[0] == "waypoints":
            gscene.show_motion(snode_nxt.traj)
            if CONNECT_MOBILE:
                to_state.Q = move_mobile_update_state(sock_mobile, MOBILE_IP, wayframer, from_state.Q, to_state.Q, 
                                                      D_APPROACH=D_APPROACH)
            raise(RuntimeError("Refine bed"))
        else:
            to_state.Q[:6] = from_state.Q[:6]
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)


motions: 3
['waypoints']
move to VIEW_MOVED_EXT: [ 3.901  1.698 -1.792  0.    -0.    -0.     0.    -0.873  1.222 -0.
  1.571 -3.142]
curre  xyzw: [ 3.23 -1.41 -0.73  0.69]
ready  xyzw: [ 3.49 -2.69  1.   -0.  ]
target xyzw: [ 3.09 -2.69  1.   -0.  ]
move to: [ 3.49 -2.69  1.   -0.  ]
(3.488006230527181, -2.689038141066413, 0.9999999737282582, -0.00019894642472386413, 0)
move to: [ 3.09 -2.69  1.   -0.  ]
(3.0880062612195927, -2.6891972978778007, 0.9999999737282582, -0.00019894642472386413, 0)
(3.0880062612195927, -2.6891972978778007, 0.9999999737282582, -0.00019894642472386413, 0)
(3.0880062612195927, -2.6891972978778007, 0.9999999737282582, -0.00019894642472386413, 0)
stop at: [ 3.11 -2.67  1.    0.01]
ended in VIEW_MOVED_EXT: [ 3.885  1.728 -1.808  0.    -0.    -0.     0.    -0.873  1.222 -0.
  1.571 -3.142]


RuntimeError: Refine bed

### 1.4 Clear highlight

In [ ]:
# test_fun.clear()
# for swp_fin in swp_fin_list:
#     gscene.remove(swp_fin)
# swp_fin_list = []
# pscene.clear_subjects()
# for child in copy.copy(bed_mat.children):
#     gscene.remove(gscene.NAME_DICT[child])


## 2. Closet cleaning

### 2.2. Make closet cleaning plan

#### 2.2.1 make plans

In [ ]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [39]:
try:
    Q_CUR = snode_schedule_list[-1][-1].state.Q
except:
    Q_CUR = HOME_POSE_SWEEP

In [ ]:
HOME_POSE_MOVE = VIEW_POSE
HOME_POSE_SWEEP = VIEW_MOVED_EXT
HOME_POSE_SWEEP[6:] = HOME_POSE_MOVE
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    


wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))

brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(0,0,-gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
                                  rpy=(0,0,0))

In [25]:


ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_leftup, brush_face, robot_config, 
                                          plane_val=None, tip_dir="up", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_leftup, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=1)
test_fun_cl.clear()
idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
snode_schedule_list_leftup, scene_args_list_leftup, scene_kwargs_list_leftup = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Q_CUR, 
    floor_ws, wayframer, closet_leftup, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)
test_fun_cl.clear()

Use 36/36 agents
======================= terminated 28: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 2: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 1: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 27: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 13: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 15: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 22: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 21: max iteration time reached (1633684937/1633684934.48 s) =

======================= terminated 32: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 24: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 31: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 34: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 5: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 16: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 18: max iteration time reached (1633684937/1633684934.48 s) ===============================
======================= terminated 3: max iteration time reached (1633684937/1633684934.48 s) ==================

======================= terminated 0: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 31: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 24: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 10: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 29: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 28: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 11: max iteration time reached (1633684943/1633684940.03 s) ===============================
======================= terminated 32: max iteration time reached (1633684943/1633684940.03 s) =================

======================= terminated 1: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 28: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 26: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 9: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 8: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 12: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 16: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 4: max iteration time reached (1633684948/1633684945.07 s) ====================

======================= terminated 3: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 29: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 5: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 15: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 35: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 18: max iteration time reached (1633684948/1633684945.07 s) ===============================
======================= terminated 25: max iteration time reached (1633684948/1633684945.07 s) ===============================
Use 36/36 agents
======================= terminated 23: max iteration time reached (1633684953/1633684950.08 s) =

...

======================= terminated 15: max iteration time reached (1633685027/1633685023.74 s) ===============================
======================= terminated 23: max iteration time reached (1633685027/1633685023.74 s) ===============================
======================= terminated 32: max iteration time reached (1633685027/1633685023.74 s) ===============================
======================= terminated 8: max iteration time reached (1633685027/1633685023.74 s) ===============================
Use 36/36 agents
Goal reached
======================= terminated 2: first answer acquired ===============================
======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 22: firs

Goal reached
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 29: first answer acquired from other agent ===============================
======================= terminated 28: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired ===============================
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 20: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired from other agent ===============================
======================= terminated 30: first answer acquired from other agent ===============================
======================= terminated 31: first answer acquired from other agent ===============================
===============

======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired from other agent ===============================
======================= terminated 25: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired ===============================
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired from other agent ===============================
Goal reached
======================= terminated 9: first answer acquired ===============================
======================= terminated 4: first answer acquired from other agent ===============================
======================= terminated 10

======================= terminated 6: first answer acquired ===============================
======================= terminated 13: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired from other agent ===============================
======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 0: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired from other agent ===============================
Goal reached
======================= terminated 5: first answer acquired from other agent ===============================
=====================

======================= terminated 34: first answer acquired from other agent ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 14: first answer acquired from other agent ===============================
======================= terminated 35: first answer acquired from other agent ===============================
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 20: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
Goal reach

In [39]:
Q_CUR = snode_schedule_list_leftup[-1][-1].state.Q

In [27]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_rightup, brush_face, robot_config, 
                                          plane_val=None, tip_dir="up", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_rightup, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=1)
test_fun_cl.clear()
idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)

snode_schedule_list_rightup, scene_args_list_rightup, scene_kwargs_list_rightup = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Q_CUR, 
    floor_ws, wayframer, closet_rightup, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)
test_fun_cl.clear()

Use 36/36 agents
Goal reached
======================= terminated 1: first answer acquired ===============================
Goal reached
======================= terminated 10: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 22: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired ===============================
======================= terminated 20: first answer acquired from other agent ===============================
======================= terminated 0: first answer acquired from other agent ===============================
Goal reached
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent =======================

======================= terminated 9: first answer acquired ===============================
======================= terminated 30: first answer acquired from other agent ===============================
======================= terminated 31: first answer acquired from other agent ===============================
======================= terminated 11: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 32: first answer acquired from other agent ===============================
======================= terminated 14: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired from other agent ===============================
======================= term

In [39]:
Q_CUR = snode_schedule_list_rightup[-1][-1].state.Q

In [28]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_down, brush_face, robot_config, 
                                          plane_val=None, tip_dir="down", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_down, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=-1)
test_fun_cl.clear()

idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list_down, scene_args_list_down, scene_kwargs_list_down = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Q_CUR, 
    floor_ws, wayframer, closet_down, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)
test_fun_cl.clear()

Use 36/36 agents
======================= terminated 8: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 17: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 1: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 19: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 35: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 32: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 3: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 21: max iteration time reached (1633685060/1633685057.5 s) ==========

======================= terminated 24: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 13: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 4: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 14: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 34: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 11: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 15: max iteration time reached (1633685060/1633685057.5 s) ===============================
======================= terminated 33: max iteration time reached (1633685060/1633685057.5 s) =========================

======================= terminated 1: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 25: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 18: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 28: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 11: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 8: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 34: max iteration time reached (1633685066/1633685063.11 s) ===============================
======================= terminated 2: max iteration time reached (1633685066/1633685063.11 s) ===================

======================= terminated 23: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 22: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 4: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 18: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 12: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 35: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 25: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 11: max iteration time reached (1633685071/1633685068.16 s) =================

======================= terminated 14: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 31: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 17: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 9: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 6: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 24: max iteration time reached (1633685071/1633685068.16 s) ===============================
======================= terminated 30: max iteration time reached (1633685071/1633685068.16 s) ===============================
Use 36/36 agents
======================= terminated 0: max iteration time reached (1633685076/1633685073.29 s) ==

...

======================= terminated 24: max iteration time reached (1633685097/1633685094.26 s) ===============================
======================= terminated 8: max iteration time reached (1633685097/1633685094.26 s) ===============================
======================= terminated 4: max iteration time reached (1633685097/1633685094.26 s) ===============================
======================= terminated 9: max iteration time reached (1633685097/1633685094.26 s) ===============================
======================= terminated 16: max iteration time reached (1633685097/1633685094.26 s) ===============================
Use 36/36 agents
======================= terminated 25: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 16: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 19: max iteration time reached (1633685102/1633685099.36 s) ==

======================= terminated 22: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 2: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 26: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 29: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 23: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 11: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 30: max iteration time reached (1633685102/1633685099.36 s) ===============================
======================= terminated 8: max iteration time reached (1633685102/1633685099.36 s) ==================

======================= terminated 4: first answer acquired from other agent ===============================
Goal reached
======================= terminated 7: first answer acquired ===============================
Goal reached
======================= terminated 12: first answer acquired ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 22: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 28: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired from other agent ===============================
Goal reached
======================= terminated 11: first answer acquired ===============================
Goal reached
===========

Goal reached
======================= terminated 5: first answer acquired ===============================
Goal reached
======================= terminated 9: first answer acquired ===============================
Use 36/36 agents
Goal reached
======================= terminated 32: first answer acquired from other agent ===============================
======================= terminated 20: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired ===============================
======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 31: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired from other agent ===============================
========

Goal reached
Goal reached
======================= terminated 29: first answer acquired ===============================
======================= terminated 9: first answer acquired ===============================
Goal reached
======================= terminated 23: first answer acquired ===============================
Goal reached
Goal reached
======================= terminated 24: first answer acquired ===============================
======================= terminated 3: first answer acquired ===============================
======================= terminated 18: first answer acquired from other agent ===============================
Goal reached
======================= terminated 26: first answer acquired ===============================
Goal reached
======================= terminated 8: first answer acquired ===============================
Goal reached
======================= terminated 2: first answer acquired ===============================
Goal reached
======================= terminate

In [29]:
snode_schedule_list = snode_schedule_list_leftup + snode_schedule_list_rightup + snode_schedule_list_down
scene_args_list = scene_args_list_leftup + scene_args_list_rightup + scene_args_list_down
scene_kwargs_list = scene_kwargs_list_leftup + scene_kwargs_list_rightup + scene_kwargs_list_down

### 2.3 Execute closet cleaning sequence

In [34]:
D_APPROACH = 0.4
UPDATE_MOTION = False
VEL_LEVEL = 1

with indy:
    vel_level_bak = indy.get_joint_vel_level()
    print("vel_level_bak: {}".format(vel_level_bak))
    
with indy:
    indy.set_joint_vel_level(VEL_LEVEL)

Connect: Server IP (192.168.0.3)
vel_level_bak: 5
Connect: Server IP (192.168.0.3)


In [31]:
swp_fin_list = []
i_s, (snode_schedule, sargs, skwargs) = list(enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)))[0]
print("motions: {}".format(len(snode_schedule[:-1])-1))
set_base_sweep(*sargs, **skwargs)


motions: 3


In [32]:
snode_pre, snode_nxt = zip(snode_schedule[:-1], snode_schedule[1:])[0]
snode_pre = snode_pre.copy(pscene)
snode_pre.traj = None
from_state = snode_pre.state
to_state = snode_nxt.state
subjects, ok = pscene.get_changing_subjects(from_state, to_state)
print(subjects)

['waypoints']


In [35]:


if len(subjects) ==0 or subjects[0] == "sweep":
    to_state.Q[:6] = from_state.Q[:6]
    if CONNECT_INDY:
        if UPDATE_MOTION:
            print("try update trajectory")
            traj, state_next, error, succ = \
                    ppline.test_connection(from_state=snode_pre.state, 
                                           to_state=snode_nxt.state)
            if succ:
                snode_nxt.traj = traj
                snode_nxt.state = state_next
    #         else:
    #             raise(RuntimeError("Path update fail"))
        ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
        with indy:
            time.sleep(0.5)
            indy.wait_for_move_finish()
    else:
        ppline.play_schedule([snode_pre, snode_nxt])

elif subjects[0] == "waypoints":
    gscene.show_motion(snode_nxt.traj)
    if CONNECT_MOBILE:
        to_state.Q = move_mobile_update_state(sock_mobile, MOBILE_IP, wayframer, from_state.Q, to_state.Q, 
                                              D_APPROACH=D_APPROACH)
else:
    to_state.Q[:6] = from_state.Q[:6]

move to VIEW_MOVED_EXT: [ 4.155e+00  2.857e+00 -1.860e-01 -1.000e-03 -0.000e+00 -0.000e+00
  0.000e+00 -8.730e-01  1.222e+00  0.000e+00  1.571e+00 -3.142e+00]
curre  xyzw: [ 3.26 -1.38 -0.72  0.69]
ready  xyzw: [ 1.03 -1.29  0.71 -0.04]
target xyzw: [ 0.63 -1.27  0.71 -0.04]
move to: [ 1.03 -1.29  0.71 -0.04]
(1.0271155144478468, -1.288814177598539, 0.7060056704643238, -0.042463419556862385, 0)
move to: [ 0.63 -1.27  0.71 -0.04]
(0.6327951018355988, -1.271858273185069, 0.7060056704643238, -0.042463419556862385, 0)


KeyboardInterrupt: 

In [ ]:

# leave highlight on cleared area
swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
gscene.update_marker(swp_fin)
swp_fin_list.append(swp_fin)

In [30]:
swp_fin_list = []

for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    print("motions: {}".format(len(snode_schedule[:-1])-1))
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        snode_pre = snode_pre.copy(pscene)
        snode_pre.traj = None
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)
        print(subjects)

        if len(subjects) ==0 or subjects[0] == "sweep":
            to_state.Q[:6] = from_state.Q[:6]
            if CONNECT_INDY:
                if UPDATE_MOTION:
                    print("try update trajectory")
                    traj, state_next, error, succ = \
                            ppline.test_connection(from_state=snode_pre.state, 
                                                   to_state=snode_nxt.state)
                    if succ:
                        snode_nxt.traj = traj
                        snode_nxt.state = state_next
            #         else:
            #             raise(RuntimeError("Path update fail"))
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt])

        elif subjects[0] == "waypoints":
            gscene.show_motion(snode_nxt.traj)
            if CONNECT_MOBILE:
                to_state.Q = move_mobile_update_state(sock_mobile, MOBILE_IP, wayframer, from_state.Q, to_state.Q, 
                                                      D_APPROACH=D_APPROACH)
        else:
            to_state.Q[:6] = from_state.Q[:6]
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)


[0.32, 0.08]
[-0.245   0.0875]
[-0.245  -0.0875]
1
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [ 0.   -3.1  -0.7   0.12]
target xyzw: [ 0.   -3.27 -0.7   0.12]
move to: [ 0.   -3.27 -0.7   0.12]
move to: [ 0.   -3.27 -0.7   0.12]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
[0.32, 0.08]
[0.     0.0875]
[ 0.     -0.0875]
1
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.5  0.2 -0.   1. ]
target xyzw: [-0.   0.2 -0.   1. ]
move to: [-0.   0.2 -0.   1. ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
[0.32, 0.08]
[0.245  0.0875]
[ 0.245  -0.0875]
1
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.68 -0.08 -0.    1.  ]
target xyzw: [-0.18 -0.08 -0.    1.  ]
move to: [-0.18 -0.08 -0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
[0.32, 0.08]
[0.49   0.0875]
[ 0.49   -0.0875]
1
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.64 -0.12 -0.    1.  ]
target xyzw: [-0.14 -0.12 -0.    1.  ]
move to: [-0.14 -0.12 -0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2

In [31]:
test_fun_cl.clear()
for swp_fin in swp_fin_list:
    gscene.remove(swp_fin)
swp_fin_list = []
pscene.clear_subjects()
for child in copy.copy(closet_leftup.children):
    gscene.remove(gscene.NAME_DICT[child])
for child in copy.copy(closet_rightup.children):
    gscene.remove(gscene.NAME_DICT[child])
for child in copy.copy(closet_down.children):
    gscene.remove(gscene.NAME_DICT[child])

## 3. Return to home

### backup - close shot bed

In [21]:
gargs = gscene.get_gtem_args()

In [24]:
for garg in gargs:
    gscene.create_safe(**garg)

In [27]:
VIEW_BED_CLOSE = np.deg2rad([0, -50, 70, -40, 100, -180])
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_BED_CLOSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        time.sleep(0.5)
        
if not CONNECT_CAM:
    import pickle

    with open(SAVE_DIR + '/bed_close_shots/bed_close_0.pkl', 'rb') as f:
        data = pickle.load(f)

    Q = data["Q"]
    T_ = data["T_robot_bed"]

    color_instance = data["image_dict"]["color"]
    depth_instance = data["image_dict"]["depth"]

    pcd = make_pcd_from_rgbd(color_instance, depth_instance)
    origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2,origin=(0,0,0))
    o3d.visualization.draw_geometries([pcd, origin])

close_view_full = np.copy(to_state.Q)
close_view_full[6:] = VIEW_BED_CLOSE
rdict = stream_capture_image(ImageType.CloseView, obj_type="bed", host=CAM_HOST)

Connect: Server IP (192.168.0.3)
Move finished!


In [29]:
SAVE_DIR

'/home/jhkim/Projects/rnb-planning/src/scripts/milestone_202110/save_img'

In [34]:
save_pickle(os.path.join(SAVE_DIR, "bed_close_shot_1008.pkl"), 
            {"rdict": rdict, "gargs": gargs, "Q": close_view_full})

In [251]:
VIEW_BED_CLOSE = np.deg2rad([0, -50, 70, -40, 100, -180])
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_BED_CLOSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        time.sleep(0.5)
        
if not CONNECT_CAM:
    import pickle

    with open(SAVE_DIR + '/bed_close_shots/bed_close_0.pkl', 'rb') as f:
        data = pickle.load(f)

    Q = data["Q"]
    T_ = data["T_robot_bed"]

    color_instance = data["image_dict"]["color"]
    depth_instance = data["image_dict"]["depth"]

    pcd = make_pcd_from_rgbd(color_instance, depth_instance)
    origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2,origin=(0,0,0))
    o3d.visualization.draw_geometries([pcd, origin])

close_view_full = np.copy(to_state.Q)
close_view_full[6:] = VIEW_BED_CLOSE
rdict = stream_capture_image(ImageType.CloseView, obj_type="bed", host=CAM_HOST)

Qdict_scan = list2dict(close_view_full, gscene.joint_names)
gscene.show_pose(Qdict_scan)
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(to_state.Q[crob.idx_dict[ROBOT_NAME]]))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        
T_bc = viewpoint.get_tf(Qdict_scan)
T_bs = bed_vis.get_tf(Qdict_scan)
# T_bs2 = np.matmul(T_bs, SE3(np.identity(3), [0.0,0, -0.0]))
T_bs2 = np.matmul(T_bs, SE3(np.identity(3), [0.2,0, -0.1]))
T_sc = np.matmul(SE3_inv(T_bs2), T_bc)
bed_dims = bed_mat.dims
floor_margin = 0.1

T_toff_bed = np.identity(4)
T_toff_bed[:3,:3] = np.array([[0,1,0],[0,0,1],[1,0,0]])
T_toff_bed[:3,3] = np.array([0.455,0,1.05])

# 실험할때 pcd 빼도록 함수 수정, pcd 경로 주석풀기
ICP_result_bed_close = reprocess_bed_detection(
    T_sc, bed_dims, floor_margin, T_toff_bed, visualize=True)

T_co = np.matmul(ICP_result_bed_close, T_toff_bed)
T_bc = viewpoint.get_tf(Qdict_scan)
T_bo = np.matmul(T_bc, T_co)

bed_center = T_bo[:3,3]
bed_rpy = Rot2rpy(T_bs[:3,:3])

# bed_center = (2,0,0)
# bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

T_revis = np.identity(4)
T_revis[:3,:3] = Rot_axis(3, Rot2axis(bed_mat.get_tf(Qdict_scan)[:3,:3],3))
bed_rpy = Rot2rpy(T_revis[:3,:3])

# adjust
bed_center[2]=0
if Rot_rpy(bed_rpy)[0,0] > 0:
    bed_rpy[2] += np.pi

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

Connect: Server IP (192.168.0.3)
Move finished!
Connect: Server IP (192.168.0.3)
Move finished!
